In [177]:
import numpy as np
import matplotlib.pyplot as plt
import os

import dicom2nifti

import gzip
import shutil

import nibabel as nib
from PIL import Image

from glob import glob
from tqdm import tqdm

## DICOM to NIFTI.GZ conversion

In [174]:
def create_nifti_CTscans_compressed(nifti_main_path, CTscans_main_path):
    # create the path list for all patient folders
    main_paths = glob(str(CTscans_main_path + r'\*'))
    # create the path list for all patient complete foot masks folders
    sub_paths = []
    for path in main_paths:
        tmp = glob(str(path + r'\*'))
        sub_paths.append(tmp[0])
    
    # iterate over all the patients for the masks
    print("Creating the .nii.gz files:")
    for i in tqdm(range(len(sub_paths))):
        # get the patient index
        idx = main_paths[i].split('\\')[-1].split('_')[-1]
        # create the path for the new .nii file using the patient index
        nifti_path = nifti_main_path + r'\patient_' + str(idx)
        # create the .nii file
        dicom2nifti.dicom_series_to_nifti(sub_paths[i], nifti_path)
        # create the .nii.gz compressed file
        with open(str(nifti_path + '.nii'), 'rb') as f_input:
            with gzip.open(str(nifti_path + '.nii.gz'), 'wb') as f_output:
                shutil.copyfileobj(f_input, f_output)
        # remove the .nii file to gain memory
        os.remove(str(nifti_path + '.nii'))

define the paths for DICOM to NIFTI conversion

In [175]:
# input
CTscans_main_path = r'C:\Users\arthu\Downloads\data\NN_Pieds'
# output
nifti_main_path = r'C:\Users\arthu\Downloads\nnUNet_raw_data_base\nnUNet_raw_data\Task501_Foot\imagesTr'

perform DICOM to NIFTI conversion

In [176]:
create_nifti_CTscans_compressed(nifti_main_path=nifti_main_path, CTscans_main_path=CTscans_main_path)

Creating the .nii.gz files:


100%|██████████| 11/11 [16:22<00:00, 89.31s/it]


## NUMPY.ARRAY to NIFTI.GZ conversion

In [136]:
def create_nifti_masks_compressed(nifti_main_path, masks_main_path):
    # create the path list for all patient folders
    main_paths = glob(str(masks_main_path + r'\*'))
    # create the path list for all patient complete foot masks folders
    sub_paths = []
    for path in main_paths:
        tmp = glob(str(path + r'\*\ALL'))
        sub_paths.append(tmp[-1])
    
    # create the path list for all files of all patient complete foot masks
    file_paths = []
    for path in sub_paths:
        file_paths.append(glob(str(path + r'\*.png')))
    
    # get the image arrays of all the patients' masks for the complete foot
    print("Loading the masks:")
    masks = []
    for idx in tqdm(range(len(file_paths))):
        foot = []
        for path in file_paths[idx]:
            foot.append(np.array(Image.open(path)))
        masks.append(np.array(foot))

    # iterate over all the patients for the masks
    print("Creating the .nii.gz files:")
    for i in tqdm(range(len(masks))):
        # get the patient index
        idx = main_paths[i].split('\\')[-1].split('_')[-1]
        # create the path for the new .nii.gz file using the patient index
        nifti_path = nifti_main_path + r'\patient_' + str(idx) + r'.nii.gz'
        # create the .nii.gz file
        nifti_img = nib.Nifti1Image(masks[i], np.eye(4))
        # save the .nii.gz file
        nib.save(nifti_img, nifti_path)

define the paths for NUMPY.ARRAY to NIFTI conversion

In [129]:
# input
masks_main_path = r'C:\Users\arthu\Downloads\data\NN_Pieds'
# output
nifti_main_path = r'C:\Users\arthu\Downloads\nnUNet_raw_data_base\nnUNet_raw_data\Task501_Foot\labelsTr'

perform NUMPY.ARRAY to NIFTI conversion

In [137]:
create_nifti_masks_compressed(nifti_main_path=nifti_main_path, masks_main_path=masks_main_path)

Loading the masks:


100%|██████████| 11/11 [00:10<00:00,  1.03it/s]


Creating the .nii.gz files:


100%|██████████| 11/11 [00:35<00:00,  3.25s/it]
